# category 数据(Categorical Data)
pandas从0.15版开始提供分类数据类型，用于表示统计学里有限且唯一性数据集。

通常实时的数据包括重复的文本列。例如：性别，国家和代码等特征总是重复的。这些是分类数据的例子。分类变量只能采用有限的数量，而且通常是固定的数量。除了固定长度，分类数据可能有顺序，但不能执行数字操作。 分类是Pandas数据类型。

分类数据的所有值都是类别或np.nan。顺序由类别的顺序定义，而不是值的词汇顺序。在内部，数据结构由类别数组和整数代码数组组成，这些代码指向类别数组中的实际值。

分类数据类型在以下情况下非常有用：

* 一个字符串变量，只包含几个不同的值。将这样的字符串变量转换为分类变量将会节省一些内存。
* 变量的词汇顺序与逻辑顺序("one"，"two"，"three")不同。 通过转换为分类并指定类别上的顺序，排序和最小/最大将使用逻辑顺序，而不是词法顺序。
* 作为其他python库的一个信号，这个列应该被当作一个分类变量(例如，使用合适的统计方法或plot类型)。

当DataFrame的某列(字段)上的数据值是都是某有限个数值的集合里的值的时候，例如：性别就男和女，有限且唯一。这列可以采用Categorical Data类型来存储、统计。

pandas的Categorical Data类型灵感来源于Data wareHorsing数据仓库里的维度表设计理念，即某列数据存储的不是数据本身，而是该数据对应的编码(有称为分类、字典编码) 这些编码比数据本身存储依赖的空间小，但能基于编码统计汇总的速度要比数据本身的存储、统计速度要快。

In [28]:
import pandas as pd
import numpy as np

## 理解category
#### 如何理解Categorical Data？
下面看一张某水果超市的供货商表(表1)：

| 供货商 | 水果   | 价格 |
| ------ | ------ | ---- |
| 1      | apple  | 5.20 |
| 2      | pearl  | 3.50 |
| 3      | orange | 7.30 |
| 5      | apple  | 5.00 |
| 6      | orange | 7.50 |
| 7      | orange | 7.30 |
| 9      | apple  | 5.20 |
| 4      | pearl  | 3.70 |
| 8      | orange | 7.30 |

第2列是各个水果供应商的能供应的水果类型，目前市场也就`apple、pearl、orange`三种水果可以买到，对于一个大超市而言可能这个表很长、有很多的水果供应商，假设有1亿条数据，那么数据存储所需空间主要浪费在水果名字上了，其他字段都是数值型的数据，而水果这一列是字符串型的，很占空间，如何能降低这张大表的存储空间浪费呢？ 设计一个辅助的水果编码表(表2)：

| 编码 | 水果   |
| ---- | ------ |
| 0    | apple  |
| 1    | pearl  |
| 2    | orange |

那么供应商的表就变为(表3)：

| 供货商 | 水果 | 价格 |
| ------ | ---- | ---- |
| 1      | 0    | 5.20 |
| 2      | 1    | 3.50 |
| 3      | 2    | 7.30 |
| 5      | 0    | 5.00 |
| 6      | 2    | 7.50 |
| 7      | 2    | 7.30 |
| 9      | 0    | 5.20 |
| 4      | 1    | 3.70 |
| 8      | 2    | 7.30 |

变化后的表的数据存储所需的空间量就下来了。也就是说在供应商表里存储的不是水果名数据本身而是存储的水果对应的编码值(通常用整形数据)。可以查供应商表里水果的编码再查辅助的编码表找到水果名。这个水果的编码表在数据仓库里称为维度表(dimension tables)。 而pandas的categorical data的作用就是构建并依赖这个维度表，即例子里的水果编码表。pandas里维度表里记录着若干且唯一的几个分类，可以通过categorical数据的**categories** 属性获得而数据的所一一对应的编码可以通过**codes**获得。


| 编码 | 水果   |
| ---- | ------ |
| 0    | apple  |
| 1    | pearl  |
| 2    | orange |

当DataFrame里的某列数据采用categorical Data方式，那么这列数据的存储会大大降低。

In [2]:
idx = [1,2,3,5,6,7,9,4,8]
name = ["apple","pearl","orange", "apple","orange","orange","apple","pearl","orange"]
price = [5.20,3.50,7.30,5.00,7.50,7.30,5.20,3.70,7.30]
df = pd.DataFrame({ "fruit": name , "price" : price}, index = idx)
df

,fruit,price
1,apple,5.2
2,pearl,3.5
3,orange,7.3
5,apple,5.0
6,orange,7.5
7,orange,7.3
9,apple,5.2
4,pearl,3.7
8,orange,7.3


看小内存信息：

In [3]:
df.memory_usage(deep=True)

Index     72
fruit    562
price     72
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 1 to 8
Data columns (total 2 columns):
fruit    9 non-null object
price    9 non-null float64
dtypes: float64(1), object(1)
memory usage: 216.0+ bytes


In [5]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 1 to 8
Data columns (total 2 columns):
fruit    9 non-null object
price    9 non-null float64
dtypes: float64(1), object(1)
memory usage: 706.0 bytes


各个列的类型：

In [6]:
df.dtypes

fruit     object
price    float64
dtype: object

将fruit列改为category类型：

In [7]:
df['fruit'] = df['fruit'].astype('category')
df['fruit']

1     apple
2     pearl
3    orange
5     apple
6    orange
7    orange
9     apple
4     pearl
8    orange
Name: fruit, dtype: category
Categories (3, object): [apple, orange, pearl]

In [8]:
df.dtypes

fruit    category
price     float64
dtype: object

In [9]:
df.memory_usage(deep=True)

Index     72
fruit    276
price     72
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 1 to 8
Data columns (total 2 columns):
fruit    9 non-null category
price    9 non-null float64
dtypes: category(1), float64(1)
memory usage: 257.0 bytes


In [11]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 1 to 8
Data columns (total 2 columns):
fruit    9 non-null category
price    9 non-null float64
dtypes: category(1), float64(1)
memory usage: 420.0 bytes


df的内存由706.0 bytes变为420.0 bytes，尽管原始的DataFrame数据量不大，所以变化比率也不大。可以适当加大df的数据长度，可以看到很明显的存储容量的降低。

In [12]:
N = 100000
df = pd.DataFrame({ "fruit": name * N, "price" : price * N}, index = idx * N)
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900000 entries, 1 to 8
Data columns (total 2 columns):
fruit    900000 non-null object
price    900000 non-null float64
dtypes: float64(1), object(1)
memory usage: 67.3 MB


In [13]:
df['fruit'] = df['fruit'].astype('category')
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900000 entries, 1 to 8
Data columns (total 2 columns):
fruit    900000 non-null category
price    900000 non-null float64
dtypes: category(1), float64(1)
memory usage: 14.6 MB


#### 理解category
总结一下pandas的category数据，两次显示DataFrame数据df的结果都是一样的，但是第二次显示的df是其fruit列经语句df`['fruit'] = df['fruit'].astype('category')`改变了其数据类型已不是object而是category类型，该列存储所需的内存使用容量大大降低。

In [14]:
N = 1
df = pd.DataFrame({ "fruit": name * N, "price" : price * N}, index = idx * N)
df['fruit'] = df['fruit'].astype('category')
df

,fruit,price
1,apple,5.2
2,pearl,3.5
3,orange,7.3
5,apple,5.0
6,orange,7.5
7,orange,7.3
9,apple,5.2
4,pearl,3.7
8,orange,7.3


fruit列是category类型的，通过codes和categorie组合出fruit的values。

In [15]:
df.fruit.values

[apple, pearl, orange, apple, orange, orange, apple, pearl, orange]
Categories (3, object): [apple, orange, pearl]

In [16]:
df.fruit.values.codes

array([0, 2, 1, 0, 1, 1, 0, 2, 1], dtype=int8)

In [17]:
df.fruit.values.categories

Index(['apple', 'orange', 'pearl'], dtype='object')

values对应于表1里的第2列即显示输出时“水果”，codes对应于表3的第2列即存储时“水果”列，categories对应于表2的“水果”列即有限唯一的一个集合。

#### 总结
Categorical Data数据由codes和categories组成，categories是有限且唯一的分类集合，codes是原数据对应的分类的编码, Categorical Data要求有限并唯一

## Category用法简介
### 创建Category对象
#### 使用astype()函数转换成category

In [18]:
s = pd.Series(["a","b","c","a"])
s

0    a
1    b
2    c
3    a
dtype: object

In [19]:
s.describe()

count     4
unique    3
top       a
freq      2
dtype: object

In [20]:
s = s.astype('category')
s

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

In [21]:
s.describe()

count     4
unique    3
top       a
freq      2
dtype: object

#### 指定dtype

In [22]:
s = pd.Series(["a","b","c","a"], dtype="category")
s

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

In [23]:
s.describe()

count     4
unique    3
top       a
freq      2
dtype: object

#### pd.Categorical构造函数
Category的构造函数：
```
pandas.Categorical(values, categories, ordered)
```

只有值参数：

In [24]:
cat = pd.Categorical(['a', 'b', 'c', 'a', 'b', 'c'])
cat

[a, b, c, a, b, c]
Categories (3, object): [a, b, c]

值参数+类别参数：

In [25]:
cat=pd.Categorical(['a','b','c','a','b','c','d'], ['c', 'b', 'a'])
cat

[a, b, c, a, b, c, NaN]
Categories (3, object): [c, b, a]

第二个参数是类别参数，在类别参数中不存在的值视为NaN

加上ordered参数：

In [26]:
cat=pd.Categorical(['a','b','c','a','b','c','d'], ['c', 'b', 'a'],ordered=True)
cat

[a, b, c, a, b, c, NaN]
Categories (3, object): [c < b < a]

逻辑排序上a>b> c

### 描述
Category对象的describe()函数，返回对Category的基础信息。

In [30]:
cat = pd.Categorical(["a", "c", "c", np.nan], categories=["b", "a", "c"])
cat.describe() #似乎与Series的describe差不多

,counts,freqs
categories,,
b,0,0.00
a,1,0.25
c,2,0.50
NaN,1,0.25


### Category的属性
categories属性：

In [31]:
cat.categories

Index(['b', 'a', 'c'], dtype='object')

codes属性：

In [32]:
cat.codes

array([ 1,  2,  2, -1], dtype=int8)

ordered属性包含了Category对象的顺序：

In [33]:
cat.ordered ## 未指定顺序

False

该函数返回结果为：False，因为这里没有指定任何顺序。

### 重命名类别
重命名类别是通过将新值分配给series.cat.categories属性来完成的。

In [34]:
s = pd.Series(["a","b","c","a"], dtype="category")
s.cat.categories = ["Group %s" % g for g in s.cat.categories]
s.cat.categories

Index(['Group a', 'Group b', 'Group c'], dtype='object')

In [35]:
s

0    Group a
1    Group b
2    Group c
3    Group a
dtype: category
Categories (3, object): [Group a, Group b, Group c]

### 附加新类别
使用Categorical.add_categories()方法，可以追加新的类别

In [40]:
cat = pd.Categorical(["a", "c", "c", 'a'], categories=["b", "a", "c"])
cat = cat.add_categories("d") #参数是list
cat

[a, c, c, a]
Categories (4, object): [b, a, c, d]

In [38]:
s = pd.Series(["a","b","c","a"], dtype="category")
s.cat.add_categories(['e']) #好像不行
s

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

In [48]:
s = s.cat.add_categories([4])
s

0    a
1    b
2    c
3    a
dtype: category
Categories (4, object): [a, b, c, 4]

### 删除类别
使用Categorical.remove_categories()方法，可以删除不需要的类别。

In [52]:
s = s.cat.remove_categories([4])
s

0    a
1    b
2    c
3    a
dtype: category
Categories (4, object): [a, b, c, 5]

In [53]:
s = s.cat.remove_categories('c')
s

0      a
1      b
2    NaN
3      a
dtype: category
Categories (3, object): [a, b, 5]

#### 分类数据的比较
在三种情况下可以将分类数据与其他对象进行比较 -

* 将等号(==和!=)与类别数据相同长度的类似列表的对象(列表，系列，数组…)进行比较。  
* 当ordered==True和类别是相同时，所有比较(==，!=，>，>=，<，和<=)分类数据到另一个分类系列。  
* 将分类数据与标量进行比较。  


In [54]:
cat = pd.Series([1,2,3]).astype("category", categories=[1,2,3], ordered=True)
cat1 = pd.Series([2,2,2]).astype("category", categories=[1,2,3], ordered=True)

cat>cat1

d:\python37\lib\site-packages\IPython\core\interactiveshell.py:3267: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  exec(code_obj, self.user_global_ns, self.user_ns)


0    False
1    False
2     True
dtype: bool

In [56]:
cat = pd.Series([1, 2, 3]).astype(pd.CategoricalDtype([3, 2, 1], ordered=True))
cat_base = pd.Series([2, 2, 2]).astype(pd.CategoricalDtype([3, 2, 1], ordered=True))
cat_base2 = pd.Series([2, 2, 2]).astype(pd.CategoricalDtype(ordered=True))
cat

0    1
1    2
2    3
dtype: category
Categories (3, int64): [3 < 2 < 1]

In [57]:
cat_base

0    2
1    2
2    2
dtype: category
Categories (3, int64): [3 < 2 < 1]

In [59]:
cat_base2

0    2
1    2
2    2
dtype: category
Categories (1, int64): [2]

Comparing to a categorical with the same categories and ordering or to a scalar works:

与具有相同类别和排序的分类或标量作品相比：

In [60]:
cat > cat_base

0     True
1    False
2    False
dtype: bool

In [61]:
cat > 2

0     True
1    False
2    False
dtype: bool

Equality comparisons work with any list-like object of same length and scalars:

等式比较适用于任何长度和标量相同的类似列表的对象：

In [62]:
cat == cat_base

0    False
1     True
2    False
dtype: bool

In [63]:
cat == np.array([1, 2, 3])

0    True
1    True
2    True
dtype: bool

In [64]:
cat == 2

0    False
1     True
2    False
dtype: bool

This doesn’t work because the categories are not the same:

这不起作用，因为类别不一样：

In [65]:
try:
    cat > cat_base2
except TypeError as e:
    print("TypeError:", str(e))

TypeError: Categoricals can only be compared if 'categories' are the same. Categories are different lengths


## 例子
https://www.jianshu.com/p/20169d7f60bc